In [34]:
import pandas as pd

df = pd.read_csv("./data/train.csv").set_index("PassengerId")

df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [35]:
from sklearn.model_selection import train_test_split

X = df.loc[:,df.columns != "Survived"]
y = df["Survived"]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
from catboost import CatBoostClassifier, Pool

train_data = Pool(data=[[1, 4, 5, 6],
                        [4, 5, 6, 7],
                        [30, 40, 50, 60]],
                  label=[1, 1, -1],
                  weight=[0.1, 0.2, 0.3])

model = CatBoostClassifier(iterations=10)

model.fit(train_data)
model.score(train_data)

Learning rate set to 0.058839
0:	learn: 0.6879920	total: 814us	remaining: 7.33ms
1:	learn: 0.6815428	total: 1.67ms	remaining: 6.68ms
2:	learn: 0.6765119	total: 2.25ms	remaining: 5.25ms
3:	learn: 0.6715373	total: 2.8ms	remaining: 4.2ms
4:	learn: 0.6653022	total: 3.32ms	remaining: 3.32ms
5:	learn: 0.6591482	total: 3.71ms	remaining: 2.48ms
6:	learn: 0.6543562	total: 4.38ms	remaining: 1.88ms
7:	learn: 0.6496176	total: 5.42ms	remaining: 1.35ms
8:	learn: 0.6436669	total: 6.37ms	remaining: 707us
9:	learn: 0.6377932	total: 6.94ms	remaining: 0us


1.0

In [37]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 332 to 103
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Name      712 non-null    object 
 2   Sex       712 non-null    object 
 3   Age       572 non-null    float64
 4   SibSp     712 non-null    int64  
 5   Parch     712 non-null    int64  
 6   Ticket    712 non-null    object 
 7   Fare      712 non-null    float64
 8   Cabin     159 non-null    object 
 9   Embarked  710 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 61.2+ KB


I can see, there are some columns containing NaN values.
These can't be be handled by Catboost, so I need to fill those missing values.

In [21]:
x_train.loc[:,'Cabin'] = x_train.loc[:,'Cabin'].fillna("")
x_train.loc[:,'Embarked'] = x_train.loc[:,'Cabin'].fillna("")
x_train.loc[:,'Age'] = x_train.loc[:,'Age'].fillna(x_train["Age"].mean())

In [33]:
print(x_train.info())
print(x_train.isna().any())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 332 to 103
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    712 non-null    int64  
 1   Name      712 non-null    object 
 2   Sex       712 non-null    object 
 3   Age       712 non-null    float64
 4   SibSp     712 non-null    int64  
 5   Parch     712 non-null    int64  
 6   Ticket    712 non-null    object 
 7   Fare      712 non-null    float64
 8   Cabin     712 non-null    object 
 9   Embarked  712 non-null    object 
dtypes: float64(2), int64(3), object(5)
memory usage: 61.2+ KB
None
Pclass      False
Name        False
Sex         False
Age         False
SibSp       False
Parch       False
Ticket      False
Fare        False
Cabin       False
Embarked    False
dtype: bool


In [ ]:
I can see, that no columns contain NaN now.

In [23]:
from catboost import CatBoostClassifier, Pool

# specify categorical features, otherwise everything would have been
# handled as float and fail
categorical_features = ["Pclass", "Name", "Sex", "Ticket", "Cabin", "Embarked"]
train_data = Pool(data=x_train,
                  label=y_train,
                  cat_features=categorical_features)

model = CatBoostClassifier(iterations=10)

model.fit(train_data)
model.score(train_data)

Learning rate set to 0.5
0:	learn: 0.5562705	total: 2.34ms	remaining: 21.1ms
1:	learn: 0.4811836	total: 4.49ms	remaining: 18ms
2:	learn: 0.4421878	total: 6.5ms	remaining: 15.2ms
3:	learn: 0.4219307	total: 8.28ms	remaining: 12.4ms
4:	learn: 0.4098689	total: 10.6ms	remaining: 10.6ms
5:	learn: 0.4003620	total: 13ms	remaining: 8.68ms
6:	learn: 0.3946399	total: 14.2ms	remaining: 6.09ms
7:	learn: 0.3905194	total: 16.3ms	remaining: 4.07ms
8:	learn: 0.3890125	total: 18.5ms	remaining: 2.05ms
9:	learn: 0.3843771	total: 20.1ms	remaining: 0us


0.901685393258427

In [15]:
x_train["Sex"]

PassengerId
332      male
734      male
383      male
705      male
814    female
        ...  
107    female
271      male
861      male
436    female
103      male
Name: Sex, Length: 712, dtype: object